Both!

In [6]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
import cProfile
import re


%matplotlib inline

the spin rate of the earth changes too
start off assuming steady state, that the moon's angular velocities in the inertial frame and the earth's angular velocities in the inertial frame are the same, so in a little time dt, the bulge and moon maintain the same seperation.

Then use change in phi* and theta* for the moon to calculate its new angular velocity about the earth.
Then use conservation of angular momentum to calculate the earth's new angular velocity. 
delta is then equal to delta + or - the little angle rotated by the moon minus the little angle rotated by the earth

delta = delta0 + (omegaMoon * t - omegaEarth * t)

In the even more general case, model the evolution tidal deformation as a satellite starts off far away and comes closer. rederive the formulas, i think. 

*Implement by 1/17/2026

**Check equations with Dr. Johnston 

In [ ]:
#20x faster than numpy..?
def cross(a, b):
    return np.array([a[1]*b[2] - a[2]*b[1], a[0]*b[2] + a[2]*b[0], a[0]*b[1] - a[1]*b[0]])

def findInertiaTensor(a, b, c, m):
    #inertia tensor over a,b,c
    tensor = np.zeros((3))
    #closed form expression
    #diagonals of the tensor
    tensor[0] = m*(b**2 + c**2)/5
    tensor[1] = m*(a**2 + c**2)/5
    tensor[2] = m*(b**2 + a**2)/5
    return tensor

def findScaledInertiaTensor(a, b, c, m):
    tensor = np.zeros((3))
    #by symmetry, off-diagonals vanich
    # Tij = ri * rj * ||r||^2
    tensor[0] = 3*m*( (a**4)/35 + ((a*b)**2 + (a*c)**2)/105 )
    tensor[1] = 3*m*( (b**4)/35 + ((b*a)**2 + (b*c)**2)/105 )
    tensor[2] = 3*m*( (c**4)/35 + ((c*a)**2 + (c*b)**2)/105 )
    return tensor

def globalToLocalRotate(r, localQuaternion):
    #rotate a globally defined vector into a locally defined coordinate system
    #returns rotated vector
    q0 = localQuaternion[0]
    qv = localQuaternion[1:4]
    t = 2.0 * cross(qv, r)
    return r - q0 * t + cross(qv, t)

def localToGlobalRotate(r, localQuaternion):
    #rotate a locally defined vector into a globally defined coordinate system
    #returns rotated vector
    q0 = localQuaternion[0]
    qv = localQuaternion[1:4]
    t = 2.0 * cross(qv, r)
    return r + q0 * t + cross(qv, t)

def findTorqueVector(Rcm, ThetaCM, RcmMagnitude, theta, DELTA, m, GAMMA_R_CUBED, InertiaTensor, ScaledInertiaTensor, MU): 
    gamma = GAMMA_R_CUBED / RcmMagnitude**3
    P2 = 3 * (np.cos(theta - DELTA))**2 - 1
    kappa = np.sin(2*theta - 2 * DELTA)

    IRcm = np.multiply(InertiaTensor, Rcm)
    SIRcm = np.multiply(ScaledInertiaTensor, Rcm) 

    m5 = 5 / RcmMagnitude**5
    m7 = 17.5 / (RcmMagnitude**7)

    # USE a+b+c = sum of diagonals over 2
    #Dividing by different magnitudes because we are using unit vectors and the derived equations use location vectors. 
    torque1 = (3 * MU / RcmMagnitude**3) * cross(Rcm, IRcm)
    torque2 = (m5) * cross(Rcm, IRcm)
    torque3 = (m7) * cross(Rcm, SIRcm)
    torque4 = (-m5) * cross(ThetaCM, IRcm)
    torque5 = (-2.5 * m * (InertiaTensor[0]+InertiaTensor[1]+InertiaTensor[2]) / RcmMagnitude**7) * cross(ThetaCM, Rcm)
    torque6 = (m7) * cross(ThetaCM, SIRcm)

    return torque1 + gamma * P2 * (torque2 + torque3) + gamma * kappa * (torque4 + torque5 + torque6)



In [10]:
def ODE(t, state, G, M, m, GAMMA_R_CUBED, InertiaTensorSecondary, ScaledInertiaTensorSecondary, InertiaTensorPrimary):

    dydt = np.empty(20)
    #-----------Secondary Body-------------#
    #Used multiple times, denoted here to calculate it only once
    
    #Points to secondary relative to primary in global coordinates
    cosTheta = np.cos(state[1])
    sinTheta = np.sin(state[1])
    cosPhi = np.cos(state[2])
    sinPhi = np.sin(state[2])

    q_s = state[9:13] / np.sqrt( state[9]**2 + state[10]**2 + state[11]**2 + state[12]**2 )
    q_p = state[16:20] / np.sqrt( state[16]**2 + state[17]**2 + state[18]**2 + state[19]**2 )

    centerOfMassUnitGlobal = np.array([sinTheta*cosPhi, sinTheta*sinPhi, cosTheta])
    centerOfMassThetaUnitGlobal = np.array([-cosTheta*cosPhi, -cosTheta*sinPhi, sinTheta])

    centerOfMassUnitSecondaryLocal = globalToLocalRotate(centerOfMassUnitGlobal, q_s)

    
    #Primary's orientation relative to the inertial frame, alligned so (0,0,1) locates the tidal bulge, and the remaining vectors locate the tidal minima
    #Find position of secondary in this frame to calculate DELTA
    DELTA = np.arccos(centerOfMassUnitSecondaryLocal[2])
    kappa = np.sin(2*(state[2] - DELTA))
    
    #The torque in the body frame of the secondary
    torque = findTorqueVector(centerOfMassUnitSecondaryLocal,
                              globalToLocalRotate(centerOfMassThetaUnitGlobal, q_p),
                              state[0], state[1], DELTA, m, GAMMA_R_CUBED, 
                              InertiaTensorSecondary, ScaledInertiaTensorSecondary, MU=G*M)
    
    #Magnitude of the torque on the secondary in the inertial frame about the primary
    kinematicTorque = (GAMMA_R_CUBED * kappa / state[0]**8)

    #Kinematic evolution of the secondary in spherical coordinates
    #Secondary's mass not used because it cancels in each of these instances
    dydt[0] = state[1]
    dydt[1] = state[0] * state[3]**2 - (G*M) / state[0]**2 - (GAMMA_R_CUBED / state[0]**7) * (3 * np.cos(state[2] - DELTA)**2 - 1)
    dydt[2] = state[3]
    dydt[3] = -2 * state[1] * state[3] / state[0] - kinematicTorque
    dydt[4] = state[5]
    dydt[5] = -2 * state[1] * state[5] / state[0]

    #Evolution of dynamic angular velocity of secondary in the body frame
    dydt[6] = ((InertiaTensorSecondary[1]-InertiaTensorSecondary[2]) * state[7] * state[8] + torque[0]) / InertiaTensorSecondary[0]
    dydt[7] = ((InertiaTensorSecondary[2]-InertiaTensorSecondary[0]) * state[8] * state[6] + torque[1]) / InertiaTensorSecondary[1]
    dydt[8] = ((InertiaTensorSecondary[0]-InertiaTensorSecondary[1]) * state[6] * state[7] + torque[2]) / InertiaTensorSecondary[2]

    #Evolution of the secondary's quaternion
    dydt[9]  = -0.5 * (q_s[1]*state[6] + q_s[2]*state[7] + q_s[3]*state[8]) 
    dydt[10] =  0.5 * (q_s[0]*state[6] - q_s[3]*state[7] + q_s[2]*state[8])
    dydt[11] =  0.5 * (q_s[3]*state[6] + q_s[0]*state[7] - q_s[1]*state[8])
    dydt[12] =  0.5 * (q_s[2]*state[6] + q_s[1]*state[7] - q_s[0]*state[8])


    #-----------Primary Body-------------#
    #Torque exerted on primary is the opposite of the net torques exerted on the secondary by CoAM
    #Net torque exerted on secondary is the kinematic torque about the primary's COM (points in decreasing theta direction) and dynamic torque about secondary's COM
    net_torque = globalToLocalRotate(localToGlobalRotate(torque, q_s), q_p) #rotate secondary body -> global -> primary body
    net_torque+= globalToLocalRotate(state[0]**2 * m * kinematicTorque * centerOfMassThetaUnitGlobal, q_p)

    #Evolution of primary's angular velocity
    dydt[13] = ((InertiaTensorPrimary[1]-InertiaTensorPrimary[2]) * state[14] * state[15] - net_torque[0]) / InertiaTensorPrimary[0]
    dydt[14] = ((InertiaTensorPrimary[2]-InertiaTensorPrimary[0]) * state[15] * state[13] - net_torque[1]) / InertiaTensorPrimary[1]
    dydt[15] = ((InertiaTensorPrimary[0]-InertiaTensorPrimary[1]) * state[13] * state[14] - net_torque[2]) / InertiaTensorPrimary[2]

    #Evolution of primary's orientation
    dydt[16] = -0.5 * (q_p[1]*state[13] + q_p[2]*state[14] + q_p[3]*state[15])
    dydt[17] =  0.5 * (q_p[0]*state[13] - q_p[3]*state[14] + q_p[2]*state[15])
    dydt[18] =  0.5 * (q_p[3]*state[13] + q_p[0]*state[14] - q_p[1]*state[15])
    dydt[19] =  0.5 * (q_p[2]*state[13] + q_p[1]*state[14] - q_p[0]*state[15])

    #derivatives of state vector
    return dydt

In [33]:
numYears = 1
T = numYears * 31556952 
sampleNumber = 100
t_eval = range(0, T, int(T/sampleNumber))

#True Constants
G = 6.67 * 10**-11
MASSPRIMARY = 5.972 * 10**24
MASSSECONDARY = 7.3 * 10**22
SHEAR_MODULUS = 150 * 10**9
MEAN_PRIMARY_RADIUS = 6371 * 1000
#a < b < c
a_s = 1736 * 1000
b_s = 1738 * 1000
c_s = 1738 * 1000

a_p = 6357 * 1000
b_p = 6378 * 1000
c_p = 6378 * 1000

#Calculated Constants
mean_density = MASSPRIMARY / (4 * 3.1415 * a_p * b_p * c_p / 3)
surface_gravity = G * MASSPRIMARY / MEAN_PRIMARY_RADIUS**2
effective_rigidity = (19 * SHEAR_MODULUS) / (2 * mean_density * surface_gravity * MEAN_PRIMARY_RADIUS)
GAMMA_R_CUBED = (9 * G * MASSSECONDARY * MEAN_PRIMARY_RADIUS**5) / (4 * (1 + effective_rigidity))

InertiaTensorSecondary = findInertiaTensor(a_s, b_s, c_s, MASSSECONDARY)
ScaledInertiaTensorSecondary = findScaledInertiaTensor(a_s, b_s, c_s, MASSSECONDARY)

InertiaTensorPrimary = findInertiaTensor(a_p, b_p, c_p, MASSPRIMARY)

#-----------Initial State Specifications-------------#

#Secondary
r =  1000000
rd = 0
theta = np.pi
thetad = 100
phi = np.pi
phid = 50
omega_1 = 0
omega_2 = 0
omega_3 = 0
q_0 = 1
q_1 = 0

q_2 = 0
q_3 = 0

#Primary
omega_1_primary = 0
omega_2_primary = 0
omega_3_primary = 0
q_0_primary = 1
q_1_primary = 0
q_2_primary = 0
q_3_primary =  0

state_naught = [r, rd, theta, thetad, phi, phid, omega_1, omega_2, omega_3, q_0, q_1, q_2, q_3, omega_1_primary, omega_2_primary, omega_3_primary, q_0_primary, q_1_primary, q_2_primary, q_3_primary]

timePoints = np.linspace(0, T, sampleNumber) 

sol = integrate.solve_ivp(rtol= .01, fun=ODE, t_span=(0, T), y0=state_naught, args=(G, MASSPRIMARY, MASSSECONDARY, GAMMA_R_CUBED, InertiaTensorSecondary, ScaledInertiaTensorSecondary, InertiaTensorPrimary))

#8.3
#
print(len(sol.t))

C:\Users\aarus\AppData\Local\Temp\ipykernel_58172\4027389031.py:24: RuntimeWarning: invalid value encountered in arccos
  DELTA = np.arccos(centerOfMassUnitSecondaryLocal[2])


820
